# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.

In [20]:
import torch
from ratsnlp.nlpbook.classification import ClassificationTrainArguments
args = ClassificationTrainArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_task_name="pair-classification",
    downstream_corpus_name="klue-nli",
    downstream_corpus_root_dir='./5-2_content/Korpora',
    # downstream_model_dir="/gdrive/My Drive/nlpbook/checkpoint-paircls",
    downstream_model_dir="./5-2_content/nlpbook/checkpoint-paircls",
    batch_size=32 if torch.cuda.is_available() else 4,
    learning_rate=5e-5,
    max_seq_length=64,
    epochs=5,
    tpu_cores=0 if torch.cuda.is_available() else 8,
    seed=7,
)

# 랜덤 시드 고정
학습 재현을 위해 랜덤 시드를 고정합니다.

In [21]:
from ratsnlp import nlpbook
nlpbook.set_seed(args)

set seed: 7


# 로거 설정
메세지 출력 등을 위한 logger를 설정합니다.

In [22]:
nlpbook.set_logger(args)

INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='pair-classification', downstream_corpus_name='klue-nli', downstream_corpus_root_dir='./5-2_content/Korpora', downstream_model_dir='./5-2_content/nlpbook/checkpoint-paircls', max_seq_length=64, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=5, batch_size=32, cpu_workers=16, fp16=False, tpu_cores=0)
INFO:ratsnlp:Training/evaluation parameters ClassificationTrainArguments(pretrained_model_name='beomi/kcbert-base', downstream_task_name='pair-classification', downstream_corpus_name='klue-nli', downstream_corpus_root_dir='./5-2_content/Korpora', downstream_model_dir='./5-2_content/nlpbook/checkpoint-paircls', max_seq_length=64, save_top_k=1, monitor='min val_loss', seed=7, overwrite_cache=False, force_download=False, test_mode=False, learning_rate=5e-05, epochs=5, batc

# 말뭉치 다운로드
실습에 사용할 말뭉치를 다운로드합니다.

In [23]:
nlpbook.download_downstream_dataset(args)

INFO:ratsnlp:cache file(/media/youngwon/Neo/NeoChoi/TIL/Pytorch-DL/ratsnlp/5-2_content/Korpora/klue-nli/klue_nli_train.json) exists, using cache!
INFO:ratsnlp:cache file(/media/youngwon/Neo/NeoChoi/TIL/Pytorch-DL/ratsnlp/5-2_content/Korpora/klue-nli/klue_nli_train.json) exists, using cache!
INFO:ratsnlp:cache file(/media/youngwon/Neo/NeoChoi/TIL/Pytorch-DL/ratsnlp/5-2_content/Korpora/klue-nli/klue_nli_train.json) exists, using cache!
INFO:ratsnlp:cache file(/media/youngwon/Neo/NeoChoi/TIL/Pytorch-DL/ratsnlp/5-2_content/Korpora/klue-nli/klue_nli_dev.json) exists, using cache!
INFO:ratsnlp:cache file(/media/youngwon/Neo/NeoChoi/TIL/Pytorch-DL/ratsnlp/5-2_content/Korpora/klue-nli/klue_nli_dev.json) exists, using cache!
INFO:ratsnlp:cache file(/media/youngwon/Neo/NeoChoi/TIL/Pytorch-DL/ratsnlp/5-2_content/Korpora/klue-nli/klue_nli_dev.json) exists, using cache!


# 토크나이저 준비
토큰화를 수행하는 토크나이저를 선언합니다

In [24]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

# 학습데이터 구축
학습데이터를 만듭니다.

In [25]:
from ratsnlp.nlpbook.paircls import KlueNLICorpus
from ratsnlp.nlpbook.classification import ClassificationDataset
from torch.utils.data import DataLoader, SequentialSampler, RandomSampler
corpus = KlueNLICorpus()
train_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

INFO:ratsnlp:Loading features from cached file ./5-2_content/Korpora/klue-nli/cached_train_BertTokenizer_64_klue-nli_pair-classification [took 0.643 s]
INFO:ratsnlp:Loading features from cached file ./5-2_content/Korpora/klue-nli/cached_train_BertTokenizer_64_klue-nli_pair-classification [took 0.643 s]
INFO:ratsnlp:Loading features from cached file ./5-2_content/Korpora/klue-nli/cached_train_BertTokenizer_64_klue-nli_pair-classification [took 0.643 s]


# 테스트 데이터 구축
학습 중에 평가할 테스트 데이터를 구축합니다.

In [26]:
val_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="test",
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)


INFO:ratsnlp:Loading features from cached file ./5-2_content/Korpora/klue-nli/cached_test_BertTokenizer_64_klue-nli_pair-classification [took 0.053 s]
INFO:ratsnlp:Loading features from cached file ./5-2_content/Korpora/klue-nli/cached_test_BertTokenizer_64_klue-nli_pair-classification [took 0.053 s]
INFO:ratsnlp:Loading features from cached file ./5-2_content/Korpora/klue-nli/cached_test_BertTokenizer_64_klue-nli_pair-classification [took 0.053 s]


# 모델 초기화
프리트레인이 완료된 BERT 모델을 읽고, 문서 쌍 분류를 수행할 모델을 초기화합니다.

In [27]:
from transformers import BertConfig, BertForSequenceClassification
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=corpus.num_labels,
)

In [28]:
model = BertForSequenceClassification.from_pretrained(
        args.pretrained_model_name,
        config=pretrained_model_config,
)

Some weights of the model checkpoint at beomi/kcbert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initiali

# 학습 준비
Task와 Trainer를 준비합니다.

In [29]:
from ratsnlp.nlpbook.classification import ClassificationTask
task = ClassificationTask(model, args)

In [30]:
trainer = nlpbook.get_trainer(args)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


# 학습
준비한 데이터와 모델로 학습을 시작합니다. 학습 결과물(체크포인트)은 미리 연동해둔 구글 드라이브의 준비된 위치(`/gdrive/My Drive/nlpbook/checkpoint-paircls`)에 저장됩니다.

In [31]:
trainer.fit(
    task,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)

Missing logger folder: /media/youngwon/Neo/NeoChoi/TIL/Pytorch-DL/ratsnlp/5-2_content/nlpbook/checkpoint-paircls/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/youngwon/anaconda3/envs/nlp_env/lib/python3.9/site-packages/pytorch_lightning/core/optimizer.py:380: RuntimeWarning: Found unsupported keys in the optimizer configuration: {'scheduler'}
  rank_zero_warn(

  | Name  | Type                          | Params
--------------------------------------------------------
0 | model | BertForSequenceClassification | 108 M 
--------------------------------------------------------
108 M     Trainable params
0         Non-trainable params
108 M     Total params
435.683   Total estimated model params size (MB)


Epoch 4: 100%|██████████| 876/876 [05:31<00:00,  2.64it/s, loss=0.161, v_num=0, acc=1.000, val_loss=1.350, val_acc=0.665]  
